<a href="https://colab.research.google.com/github/jneey2000/ML/blob/main/1_neural_network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import keras
keras.__version__

'2.15.0'

In [ ]:
!nvidia-smi

Mon May 27 11:23:44 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   43C    P0              26W /  70W |   1135MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

# A first look at a neural network

This notebook contains the code samples found in Chapter 2, Section 1 of [Deep Learning with Python](https://www.manning.com/books/deep-learning-with-python?a_aid=keras&a_bid=76564dff). Note that the original text features far more content, in particular further explanations and figures: in this notebook, you will only find source code and related comments.

----

<p align="left"><img src="https://drive.google.com/uc?export=download&id=1t0vNrvVcu1dyqiYxAgKdUV7RH-kocAG5" width="800"/></p>




Flatten layer와 Dense layer의 차이:  
- Flatten layer는 다차원 입력을 1차원으로 변환하는 역할을 함
- 예를 들어 입력 크기가 (28, 28, 1)인 이미지라면 Flatten layer를 거치게 되면 (28 x 28 x 1) = 784차원의 벡터가 됨
- Flatten layer는 학습되는 파라미터가 없고, 단순히 차원만 바꿔주는 layer임

- Dense layer는 입력 벡터의 모든 뉴런을 다음 layer의 모든 뉴런에 연결해 주는 layer임
- Dense layer에서는 각각의 연결에 대해 weight과 bias를 학습함

We will now take a look at a **first concrete example of a neural network**, which makes use of the Python library Keras to learn **to classify
hand-written digits**.   
(이제 Python 라이브러리 Keras를 사용하여 손으로 쓴 숫자를 분류하는 방법을 학습하는 예를 살펴보자)

Unless you already have experience with Keras or similar libraries, you will not understand everything about this
first example right away. You probably haven't even installed Keras yet. Don't worry, that is perfectly fine.

In the next chapter, we will
review each element in our example and explain them in detail.

So don't worry if some steps seem arbitrary or look like magic to you!
We've got to start somewhere.

- The problem we are trying to solve here is **to classify grayscale images of handwritten digits (28 pixels by 28 pixels), into their 10 categories (0 to 9)**.   
(여기서 해결하려는 문제는 손으로 쓴 숫자(28픽셀 x 28픽셀)의 회색조 이미지를 10개 범주(0~9)로 분류하는 것임)

- **The dataset** we will use is the **MNIST dataset**, a classic dataset in the machine learning community, which has been
around for almost as long as the field itself and has been very intensively studied.
(우리가 사용할 데이터 세트는 기계 학습 커뮤니티에서 가장 고전적으로 사용되는 MNIST 데이터 셋임. 이 데이터 셋은 해당 분야만큼이나 오랫동안 사용되어 왔으며 매우 집중적으로 연구되어 왔음)

- It's a set of **60,000 training images**, plus **10,000 test
images**, assembled by the National Institute of Standards and Technology (the NIST in MNIST) in the 1980s.  
(이는 1980년대 미국 국립표준기술연구소(MNIST의 NIST)에서 수집한 60,000개의 훈련 이미지와 10,000개의 테스트 이미지 셋으로 구성되어 있음)

- You can think of "solving" MNIST
as the "Hello World" of deep learning -- it's what you do to verify that your algorithms are working as expected.   
(MNIST를 딥 러닝의 "Hello World"로 생각하면 됨 -- 이는 알고리즘이 예상대로 작동하는지 확인하기 위해 수행하는 작업임)

- As you become a machine
learning practitioner, you will see MNIST come up over and over again, in scientific papers, blog posts, and so on.  
(머신러닝 전문가가 되면 과학 논문, 블로그 게시물 등에서 MNIST가 계속해서 등장하는 것을 보게 될 것)

<p align="left"><img src="https://drive.google.com/uc?export=view&id=1JWgEwdPYlhItjkTX33Ic1o9MvjMdlo0o" width="600"/></p>

The MNIST dataset comes pre-loaded in Keras, in the form of a set of four Numpy arrays:

In [ ]:
from tensorflow.keras.datasets import mnist

(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

## Train dataset, Test dataset
- `train_images` and `train_labels` form the "training set", the data that the model will learn from.
- The model will then be tested on the
"test set", `test_images` and `test_labels`.
- Our images are encoded as Numpy arrays, and the labels are simply an array of digits, ranging
from 0 to 9.   
(우리의 이미지는 Numpy 배열로 인코딩되었으며 레이블은 단순히 0에서 9까지의 숫자 배열임)
- There is a one-to-one correspondence between the images and the labels.  
(이미지와 라벨 사이에는 일대일 대응이 있음)  
Let's have a look at the training data:

In [ ]:
train_images.shape

(60000, 28, 28)

In [ ]:
len(train_labels)

60000

In [ ]:
train_labels

array([5, 0, 4, ..., 5, 6, 8], dtype=uint8)

In [ ]:
train_images.dtype # 값이 0~255 범위의 정수이며 음수는 없는 8비트 정수형

dtype('uint8')

Let's have a look at the test data:

In [ ]:
test_images.shape

(10000, 28, 28)

In [ ]:
len(test_labels)

10000

In [ ]:
test_labels

array([7, 2, 1, ..., 4, 5, 6], dtype=uint8)

In [ ]:
test_images.dtype

dtype('uint8')

### Our workflow will be as follow:
- first we will present our neural network with the training data, `train_images` and `train_labels`.   
(먼저 훈련 데이터인 train_images 및 train_labels가 포함된 신경망을 제시함)  
- The
network will then learn to associate images and labels.   
(그러면 네트워크는 이미지와 라벨을 연결하는 방법을 학습함)  
- Finally, we will ask the network to produce predictions for `test_images`, and we
will verify if these predictions match the labels from `test_labels`.  
(마지막으로 네트워크에 test_images에 대한 예측을 생성하도록 요청하고 이러한 예측이 test_labels의 레이블과 일치하는지 확인함)  

Let's build our network -- again, remember that you aren't supposed to understand everything about this example just yet.

참고) Keras는 딥러닝 모델을 쉽게 만들고 학습시킬 수 있도록 도와주는 고수준(high-level) 딥러닝 API
즉, 복잡한 TensorFlow나 PyTorch 코드 대신에 직관적인 방법으로 모델을 구성할 수 있게끔 해줌. 예전에는 Theano, TensorFLow, CNTK 등을 백엔드로 사용했지만, 현재는 Tensorflow 전용으로 통합되었음.

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Model

model = keras.Sequential()
model.add(layers.Dense(512, activation='relu', input_shape=(28 * 28,)))
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))

In [ ]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_4 (Dense)             (None, 512)               401920    
                                                                 
 dense_5 (Dense)             (None, 256)               131328    
                                                                 
 dense_6 (Dense)             (None, 128)               32896     
                                                                 
 dense_7 (Dense)             (None, 10)                1290      
                                                                 
Total params: 567434 (2.16 MB)
Trainable params: 567434 (2.16 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


### Architecture of Deep Neural Networks
- The core building block of neural networks is the "**layer**", a data-processing module which you can conceive as a "filter" for data.
- Some
data comes in, and comes out in a more useful form.
- Precisely, layers extract **representations** out of the data fed into them -- hopefully **representations** that are more meaningful for the problem at hand.  
(<font color="orange">**Layer는 주어진 문제에 더 의미있는 표현(representation)을 입력된 데이터로 부터 추출**</font>)
- Most of deep learning really consists of chaining together simple layers
which will implement a form of progressive "data distillation".  
(<font color="orange">**대부분의 딥러닝은 간단한 층을 연결하여 구성되고 있고, 점진적으로 데이터를 정제하는 형태를 띠고 있음**</font>)
- A deep learning model is like a sieve for data processing, made of a
succession of increasingly refined data filters -- the "layers".  
(<font color="orange">**딥러닝 모델은 데이터 정제 필터(층)가 연속되어 있는 데이터 프로세싱을 위한 여과기와 같음**</font>)




**Example**:
- Here our network consists of a sequence of four `Dense` layers, which are densely-connected (also called "fully-connected") neural layers.   
(여기서 우리의 네트워크는 4개의 연속적인 'Dense' layers ("fully-connected"이라고도 함)로 구성됨)
- The fourth (and last) layer is <U>a 10-way "softmax" layer</U>, which means it will return an array of 10 probability scores (summing to 1).
(네 번째(마지막) 레이어에서 activation function은 softmax를 사용함. 이는 10개의 확률 점수(합계가 1이 됨)의 배열을 반환한다는 의미임)  
- Each score will be the probability that the current digit image belongs to one of our 10 digit classes.  
(각 점수는 현재 숫자 이미지가 10자리 클래스 중 하나에 속할 확률을 나타냄)


To make our network ready for training, we need to pick three more things, as part of <B>"compilation" step</B>:

* <B>A loss function</B>: this is how the network will be able to measure how good a job it is doing on its training data, and thus how it will be
able to steer itself in the right direction.  
(훈련 데이터에서 신경망의 성능을 측정하는 방법)
* <B>An optimizer</B>: this is the mechanism through which the network will update itself based on the data it sees and its loss function.  
(입력된 데이터와 손실 함수를 기반으로 네트워크를 업데이트하는 방법)
* <B>Metrics to monitor during training and testing</B>. Here we will only care about accuracy (the fraction of the images that were correctly
classified).  

In [ ]:
model.compile(optimizer='rmsprop',
                loss='categorical_crossentropy',
                metrics=['accuracy'])

### Preprocessiong
- Convert Input data shape : Before training, we will preprocess our data by reshaping it into the shape that the network expects.  
(데이터를 network에 맞는 크기로 바꿈)
- Normalization: Scaling it so that all values are in the `[0, 1]` interval.

Previously, our training images for instance were stored in an array of shape `(60000, 28, 28)` of type `uint8` with
values in the <U>`[0, 255]` interval</U>.

We transform it into a `float32` array of shape `(60000, 28 * 28)` with values <font color="orange"><U>between 0 and 1</U></font>.

즉, 이 예에서는 다음과 같이 세 가지를 변경함:  
- shape: `(60000, 28, 28)` --> `(60000, 28*28)`
- type: `uint8` -->`float32`
- Normalization: `[0, 255]` --> `[0, 1]`

In [ ]:
train_images = train_images.reshape((60000, 28 * 28))
train_images = train_images.astype('float32') / 255

test_images = test_images.reshape((10000, 28 * 28))
test_images = test_images.astype('float32') / 255

We also need to **categorically encode the labels**  (원-핫 인코딩)

In [ ]:
from keras.utils import to_categorical

train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)


We are now ready to train our network, which in Keras is done via a call to <B>the `fit` method</B> of the network:
we "fit" the model to its training data.  
(케라스에서는 fit 메서드를 호출하여 훈련 데이터에 모델을 학습 시킴)

In [ ]:
model.fit(train_images, train_labels, epochs=5, batch_size=128)

Epoch 1/5
469/469 [==============================] - 4s 3ms/step - loss: 0.2521 - accuracy: 0.9210
Epoch 2/5
469/469 [==============================] - 2s 3ms/step - loss: 0.0896 - accuracy: 0.9723
Epoch 3/5
469/469 [==============================] - 2s 3ms/step - loss: 0.0589 - accuracy: 0.9815
Epoch 4/5
469/469 [==============================] - 2s 3ms/step - loss: 0.0419 - accuracy: 0.9865
Epoch 5/5
469/469 [==============================] - 2s 3ms/step - loss: 0.0312 - accuracy: 0.9900


Two quantities are being displayed during training:
- the <B>"loss"</B> of the network over the training data
- the <B>accuracy</B> of the network over the training data.

We quickly reach an accuracy of 0.989 (i.e. 98.9%) on the training data.  

<B>Now let's check that our model performs well on the test set too</B>:  
(<font color="orange">evaluate 메서드를 통해 모델의 일반화 성능 (즉, 테스트 데이터셋의 성능)을 체크할 수 있음</font>)

In [ ]:
test_loss, test_acc = model.evaluate(test_images, test_labels)  # 디폴트 배치 수 : 32

313/313 [==============================] - 1s 4ms/step - loss: 0.0723 - accuracy: 0.9797


In [ ]:
print('test_acc:', test_acc)

test_acc: 0.9797000288963318



Our test set accuracy turns out to be 97.8% -- that's quite a bit lower than the training set accuracy.   
<font color="orange">This gap between training accuracy and test accuracy is an example of "**overfitting**"</font>,
the fact that machine learning models tend to perform worse on new data than on their training data.   
(우리의 테스트 세트 정확도는 97.8%로 나타났음. 이는 훈련 세트 정확도보다 상당히 낮은 수치임. 훈련 정확도와 테스트 정확도 사이의 이러한 격차는 "과적합"의 예임. 즉, 기계 학습 모델이 훈련 데이터보다 새 데이터에서 성능이 떨어지는 경향이 있다는 사실을 나타냄)

- This concludes our very first example -- you just saw how we could build and a train a neural network to classify handwritten digits, in
less than 20 lines of Python code.